# KNN

In [1]:
# 1. Exploratory Data Analysis
# 2. Data Preprocessing & Feature Engineering
# 3. Modeling & Prediction
# 4. Model Evaluation
# 5. Hyperparameter Optimization
# 6. Final Model

import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [2]:
# 1. Exploratory Data Analysis

df = pd.read_csv("diabetes.csv")

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.shape

(768, 9)

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [5]:
df["Outcome"].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [6]:
# 2. Data Preprocessing & Feature Engineering

y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

# knn ve gradient descent gibi uzaklık temelli yöntemlerde
# değişkenleri standartlaştırmak başarıyı artırır
X_scaled = StandardScaler().fit_transform(X)

X = pd.DataFrame(X_scaled, columns=X.columns)

In [7]:
# 3. Modeling & Prediction

knn_model = KNeighborsClassifier().fit(X, y)

In [8]:
random_user = X.sample(1, random_state=45)
random_user

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
195,0.342981,1.161295,0.770014,1.283638,1.130518,0.940144,-0.232176,-0.360847


In [9]:
knn_model.predict(random_user)

array([1], dtype=int64)

In [10]:
# 4. Model Evaluation

# Confusion matrix için y_pred:
y_pred = knn_model.predict(X)

In [11]:
# AUC için y_prob:
y_prob = knn_model.predict_proba(X)[:, 1]

In [12]:
print(classification_report(y, y_pred))
# acc 0.83
# f1 0.74
# AUC

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



In [13]:
roc_auc_score(y, y_prob)
# 0.90

0.9017686567164179

In [14]:
# cross validation
cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
cv_results

{'fit_time': array([0.00541997, 0.00299025, 0.00199318, 0.00099683, 0.00199318]),
 'score_time': array([0.02765083, 0.0139575 , 0.01196003, 0.01195979, 0.01196003]),
 'test_accuracy': array([0.72077922, 0.73376623, 0.71428571, 0.77124183, 0.7254902 ]),
 'test_f1': array([0.58252427, 0.60952381, 0.54166667, 0.63917526, 0.58      ]),
 'test_roc_auc': array([0.77555556, 0.78759259, 0.73194444, 0.83226415, 0.77528302])}

In [15]:
cv_results['test_accuracy'].mean()

0.733112638994992

In [16]:
cv_results['test_f1'].mean()

0.5905780011534191

In [17]:
cv_results['test_roc_auc'].mean()

0.7805279524807827

In [18]:
# 5. Hyperparameter Optimization

knn_model = KNeighborsClassifier()
knn_model.get_params()

# PARAMETRE: modellerin veri içinden öğrendiği ağırlıklar
# HİPERPARAMETRE: kullanıcının tanımladığı dışsal parametre

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [19]:
# optimum komşuluk sayısını bulalım

knn_params = {"n_neighbors": range(2, 50)}

knn_gs_best = GridSearchCV(knn_model,
                           knn_params,
                           cv=5,
                           n_jobs=-1,
                           verbose=1).fit(X, y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [20]:
knn_gs_best.best_params_

{'n_neighbors': 17}

In [21]:
# 6. Final Model

knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X, y)

cv_results = cross_validate(knn_final,
                            X,
                            y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])
cv_results

{'fit_time': array([0.0094924 , 0.0019927 , 0.00689745, 0.00398564, 0.00738692]),
 'score_time': array([0.03760338, 0.02148342, 0.03422618, 0.03590441, 0.02389002]),
 'test_accuracy': array([0.75324675, 0.74025974, 0.75974026, 0.81045752, 0.77124183]),
 'test_f1': array([0.60416667, 0.60784314, 0.57471264, 0.70103093, 0.59770115]),
 'test_roc_auc': array([0.80675926, 0.7912037 , 0.79166667, 0.84603774, 0.82830189])}

In [22]:
cv_results['test_accuracy'].mean()

0.7669892199303965

In [23]:
cv_results['test_f1'].mean()

0.6170909049720137

In [24]:
cv_results['test_roc_auc'].mean()

0.8127938504542278

In [25]:
random_user = X.sample(1)

knn_final.predict(random_user)

array([0], dtype=int64)